In [1]:
import numpy as np

In [26]:
a = np.array([[1,2],[3,4]])
a

array([[1, 2],
       [3, 4]])

In [27]:
b = np.array([[1,0],[0,1],[1,1]])
b

array([[1, 0],
       [0, 1],
       [1, 1]])

In [31]:
e = np.reshape(b,(3,1,2))
e

array([[[1, 0]],

       [[0, 1]],

       [[1, 1]]])

In [64]:
c = np.tile(a,(3,1))
# c

In [24]:
d = np.reshape(c,(3,2,2))
d

array([[[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]]])

In [42]:
f = d-e
f

array([[[0, 2],
        [2, 4]],

       [[1, 1],
        [3, 3]],

       [[0, 1],
        [2, 3]]])

In [60]:
np.sum(np.abs(f),axis = 2)

array([[2, 6],
       [2, 6],
       [1, 5]])

# 总结一下
broadcast可以迭代到内层，直至运算完毕。

所以想要计算(500*n) 与 (5000*n)的两个数组的三维计算，需要扩张其维度

np.reshape(b,(5000,1,n))

与

np.reshape(np.tile(a,(5000,1)),(5000,500,n))

In [50]:
def D3Broadcast(a, b):
    # a is Axn, b is Bxn
    # Return BxA
    assert b.shape[1] == a.shape[1]
    b_new = np.reshape(b,(b.shape[0],1,b.shape[1]))
    a_new = np.tile(a,(b.shape[0],1))
    a_new = np.reshape(a_new,(b.shape[0],a.shape[0],b.shape[1]))
    f = a_new-b_new
    return np.sum(np.square(f),axis = 2)

In [51]:
D3Broadcast(a,b)

array([[ 383,  228,  253,  473,  229,  411],
       [ 158,  -53,   18,  304, -176,  186],
       [ 120,  177,  394,  304,  212,  -38]])

In [63]:
D3Broadcast(b,a)

array([[2, 2, 1],
       [6, 6, 5]])

# 实测以上方法不行
`np.tile`需要过多内存

In [2]:
a_l = 6 
b_l = 3
n = 10
a = np.random.randint(-9,9,a_l*n, dtype=np.int8).reshape(a_l,n)
a.shape

(6, 10)

In [3]:
b = np.random.randint(-9,9,b_l*n, dtype=np.int8).reshape(b_l,n)
b.shape

(3, 10)

In [46]:
c = np.sum(np.square(a.reshape(a.shape[0],1,a.shape[1])-b),axis = 2)
c.shape
' DO NOT RUN THIS, MEMORY EXPLOISION'
c

array([[ 383,  158,  120],
       [ 228,  -53,  177],
       [ 253,   18,  394],
       [ 473,  304,  304],
       [ 229, -176,  212],
       [ 411,  186,  -38]])

# 实测以上方法还是不行

任然爆掉内存了

In [52]:
# del c
m = np.zeros((a.shape[0],b.shape[0],a.shape[1]))
np.subtract(a.reshape(a.shape[0],1,a.shape[1]), b, m)
ans = np.sum(np.square(m),axis = 2)
ans.shape
ans

array([[383., 414., 632.],
       [484., 459., 177.],
       [253., 274., 394.],
       [729., 304., 304.],
       [229., 592., 212.],
       [411., 186., 218.]])

# 实测还是不行

尝试 $(a-b)^2 =(a-b)·(a-b)= a^2 +b^2 -a·b - b·a$

In [39]:
a1 = np.sum(np.square(a),axis=1)
a1

array([320, 163, 202, 262, 264, 150])

In [43]:
b1 = np.transpose(np.sum(np.square(b),axis=1))
b1.transpose()
b1

array([159, 172, 202])

In [45]:
a2b2 = a1[:,np.newaxis]+b1
a2b2

array([[479, 492, 522],
       [322, 335, 365],
       [361, 374, 404],
       [421, 434, 464],
       [423, 436, 466],
       [309, 322, 352]])

In [35]:
ba = np.dot(a, b.T)
ba

array([[ 48,  39, -55],
       [-81, -62,  94],
       [ 54,  50,   5],
       [102,  65,  80],
       [ 97, -78, 127],
       [-51,  68,  67]], dtype=int8)

In [36]:
(a2b2-2*ba)

array([[383, 414, 632],
       [228, 459, 433],
       [253, 274, 394],
       [473, 560, 560],
       [485, 336, 468],
       [411, 442, 474]])

In [60]:
(-2*np.dot(a, b.T) + 
        np.sum(np.square(b), axis = 1) + 
        np.transpose([np.sum(np.square(a), axis = 1)]))[:,np.newaxis]

array([[[383, 414, 632]],

       [[228, 459, 433]],

       [[253, 274, 394]],

       [[473, 560, 560]],

       [[485, 336, 468]],

       [[411, 442, 474]]])

# 2

In [63]:
a = np.random.randint(-9,9,10, dtype=np.int8)
a_l = np.array_split(a, 4)
a_l

[array([ 7, -2,  1], dtype=int8),
 array([7, 5, 2], dtype=int8),
 array([ 6, -8], dtype=int8),
 array([-9,  7], dtype=int8)]

In [71]:
a_l. 

[array([ 7, -2,  1], dtype=int8),
 array([7, 5, 2], dtype=int8),
 array([ 6, -8], dtype=int8),
 array([-9,  7], dtype=int8)]

### 3


In [28]:
import numpy as np
scores = np.random.random((5,3))
scores

array([[0.79795733, 0.73763993, 0.25210208],
       [0.04185845, 0.24514837, 0.18982861],
       [0.86313489, 0.09919257, 0.19046188],
       [0.04778428, 0.2552938 , 0.11215196],
       [0.48997616, 0.95479607, 0.6576028 ]])

In [29]:
max4rows = np.max(scores,axis=1)
max4rows

array([0.79795733, 0.24514837, 0.86313489, 0.2552938 , 0.95479607])

In [30]:
argmax4rows = np.argmax(scores,axis=1)
argmax4rows

array([0, 1, 0, 1, 1])

In [31]:
scores[range(len(argmax4rows)),argmax4rows] = 0
scores

array([[0.        , 0.73763993, 0.25210208],
       [0.04185845, 0.        , 0.18982861],
       [0.        , 0.09919257, 0.19046188],
       [0.04778428, 0.        , 0.11215196],
       [0.48997616, 0.        , 0.6576028 ]])

In [61]:
a = np.array([[1,2], [3, 4], [5, 6]])
a

array([[1, 2],
       [3, 4],
       [5, 6]])

In [62]:
a[a > 2] = 0
a

array([[1, 2],
       [0, 0],
       [0, 0]])

In [67]:
np.c_[a, np.ones(3).reshape(3,1)]

array([[1., 2., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [66]:
np.r_[a, np.ones(2).reshape(1,2)]

array([[1., 2.],
       [0., 0.],
       [0., 0.],
       [1., 1.]])

In [52]:
np.ones(2)

array([1., 1.])

In [57]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[0,0,0]])
c = np.r_[a,b]
d = np.c_[a,b.T]
print(c,'\n',d)

[[1 2 3]
 [4 5 6]
 [7 8 9]
 [0 0 0]] 
 [[1 2 3 0]
 [4 5 6 0]
 [7 8 9 0]]


In [74]:
scores

array([[0.        , 0.73763993, 0.25210208],
       [0.04185845, 0.        , 0.18982861],
       [0.        , 0.09919257, 0.19046188],
       [0.04778428, 0.        , 0.11215196],
       [0.48997616, 0.        , 0.6576028 ]])

In [72]:
scores[:,0:-1]

array([[0.        , 0.73763993],
       [0.04185845, 0.        ],
       [0.        , 0.09919257],
       [0.04778428, 0.        ],
       [0.48997616, 0.        ]])

In [73]:
scores[:,-1]

array([0.25210208, 0.18982861, 0.19046188, 0.11215196, 0.6576028 ])

In [81]:
scores = np.random.random((5,3))-0.5
scores

array([[ 0.44964485, -0.34288448, -0.31976885],
       [ 0.02492434, -0.45917017, -0.48579053],
       [ 0.39132555,  0.15632048,  0.30745887],
       [ 0.21173017, -0.06495827, -0.3323145 ],
       [ 0.27998779, -0.46459054,  0.43615421]])

In [87]:
(scores.T - np.max(scores, axis =1)).T

array([[ 0.        , -0.79252933, -0.76941371],
       [ 0.        , -0.48409451, -0.51071487],
       [ 0.        , -0.23500507, -0.08386668],
       [ 0.        , -0.27668844, -0.54404467],
       [-0.15616642, -0.90074475,  0.        ]])

In [85]:
np.max(scores, axis =1)

array([0.44964485, 0.02492434, 0.39132555, 0.21173017, 0.43615421])